# Using Google Auto ML
This is a document that outlines how to get started on creating a custom object detector using Google's AutoML API (https://cloud.google.com/automl).


## Requirements
- A Google Cloud account in the Free Tier. See https://cloud.google.com/free/docs/gcp-free-tier
- Billing Information (Google claims it will not auto charge to card if doing a free trial. The free trial lasts until $300 credit is used up or until a year has past since sign up).
- Python. Supported versions are 3.5 to 3.7, and 2.7.9 or higher. Recommend >= 3.6 
- Linux, macOS, or Windows device 
Cloud SDK requires Python. Some tools bundled with Cloud SDK have additional requirements

## Setting Up Your Data (Photo Example)
The Google AutoML API accepts photos and screenshots from videos to train your custom object detector.

In [13]:
from google.cloud import automl
from datetime import datetime
import os
import glob
import pandas as pd

Set the following variables.

In [17]:
FILE_PATH = os.getcwd()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/SusanneGov/Documents/My First Project-bcfca3ee21b0.json" # replace with your JSON path
project_id = "micro-progress-275116"
bucket_name = "overhead-signs" # The name you want the photos to be put in Google Cloud storage
folders = ['test_photos'] # Add local folder name or multiple folders used for training data

We will create an CSV of unlabeled images from an existing folder on the local computer. For the guide, we will be using the 'test_photos' folder. The CSV columns are set to 10 columns.

<b>label,image_path,x1,y1,None,None,x2,y2,None,None</b>
- label is the label for bounding box
- image_path is the Google Cloud Storage path
- x1/x2 and y1/y2 are coordinates of bounding box
- None is simply blank column

In [30]:
png_list = [] # keep it empty 
for f in folders:
    google_url = r'gs://{0}/{1}/'.format(bucket_name,f)
    for file in glob.glob(FILE_PATH + "//" + f + '/*.png'):
        value = (google_url + os.path.basename(file),
                 '','','','','','','','','')
        png_list.append(value)  
png_df = pd.DataFrame(png_list)
png_df['set'] = 'UNASSIGNED'
output_csv = FILE_PATH + "//" + bucket_name + "-" + datetime.now().strftime("%m%d%Y") + ".csv"
png_df.set_index('set').to_csv(output_csv,header=None)

## Importing Photos and CSV file in Google Cloud Storage
After the CSV has been created, you need to upload the photos and CSV you plan to use to train your object detector onto the Google Cloud Storage https://console.cloud.google.com/storage 

1) Set up a "bucket" to put the folder(s) of photos and csv files in.

![photo](6.png)

2) After bucket has been created, we can start importing the folder containing photos and the CSV file created in Python.

![photo](.png)
labeling the photos data.

In [3]:
client = automl.AutoMlClient()

# A resource that represents Google Cloud Platform location.
project_location = client.location_path(project_id, "us-central1")
metadata = automl.types.ImageObjectDetectionDatasetMetadata()
dataset = automl.types.Dataset(
    display_name=display_name,
    image_object_detection_dataset_metadata=metadata,
)

# Create a dataset with the dataset metadata in the region.
response = client.create_dataset(project_location, dataset)

created_dataset = response.result()

# Display the dataset information
print("Dataset name: {}".format(created_dataset.name))
print("Dataset id: {}".format(created_dataset.name.split("/")[-1]))

Dataset name: projects/409520144140/locations/us-central1/datasets/IOD3508489927186710528
Dataset id: IOD3508489927186710528
